# Instalando libs

In [1]:
!pip install rank_bm25
!pip install evaluate
!pip install trectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 765.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 3.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel f

# Montando o drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importando libs

In [3]:
import numpy as np
import pandas as pd
import random
import string
import nltk
import json
import math
from evaluate import load
from rank_bm25 import BM25Okapi
import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Definindo o path dos arquivos

In [4]:
PATH = '/content/drive/MyDrive/Colab Notebooks/IR/data/'

# Lendo os arquivos

In [5]:
# Getting document set
document_set = {}
doc_id = ""
doc_text = ""

# Opening file
with open(PATH + 'CISI.ALL') as f:
    lines = ""
    
    # Iterating through lines and parsing 
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")

# Getting relevant lines
doc_count = 0
for l in lines:
    if l.startswith(".I"): # .I stands for index
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"): # .X may not be relevant
        document_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else: # Concating author, title and text
        doc_text += l.strip()[3:] + " " 


# Getting query set
query_set = {}
query_id = ""

# Openinig file
with open(PATH + 'CISI.QRY') as f:
    lines = ""

    # Iterating through lines and parsing
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# Getting relevant lines
for l in lines:
    if l.startswith(".I"): # .I stands for index
        query_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"): # .W stands for the query text
        query_set[query_id] = l.strip()[3:]
        query_id = ""


# Getting relevant set
relevant_set = {}

# Opening file
with open(PATH + 'CISI.REL') as f:

    # Iterating through lines and parsing 
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in relevant_set:
            relevant_set[qry_id].append(doc_id)
        else:
            relevant_set[qry_id] = []
            relevant_set[qry_id].append(doc_id)


In [6]:
document_set[0]

"18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. "

In [7]:
query_set[0]

'What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?'

In [8]:
relevant_set[0]

[27,
 34,
 37,
 41,
 42,
 51,
 64,
 75,
 85,
 149,
 188,
 191,
 192,
 194,
 214,
 268,
 290,
 319,
 428,
 464,
 465,
 481,
 482,
 509,
 523,
 540,
 575,
 581,
 588,
 602,
 649,
 679,
 710,
 721,
 725,
 782,
 812,
 819,
 867,
 868,
 893,
 1161,
 1163,
 1194,
 1195,
 1280]

# Lista com queries sem documentos relevantes

Existem algumas queries que não tem correspondência no arquivo de documentos relevantes. Vou deixá-las explícitas em uma variável para utilizar posteriormente. A ideia é não calcular métricas para essas queries.*texto em itálico*

In [9]:
queries_with_no_relevant_docs = np.setdiff1d(list(query_set.keys()),list(relevant_set.keys()))

## Algumas estatísticas

In [10]:
print('# de documentos:', len(document_set))
print('# de queries:', len(query_set))
print('# de mappings:', len(relevant_set))
print('# de tokens no menor documento:', np.min([len(value) for key, value in relevant_set.items()]))
print('# de tokens no maior documento:', np.max([len(value) for key, value in relevant_set.items()]))
print('Média de tokens por documento:', np.mean([len(value) for key, value in relevant_set.items()]))
print('Queries sem documentos relevantes:', queries_with_no_relevant_docs)

# de documentos: 1460
# de queries: 112
# de mappings: 76
# de tokens no menor documento: 1
# de tokens no maior documento: 155
Média de tokens por documento: 40.973684210526315
Queries sem documentos relevantes: [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]


# Limpando e criando o corpus

A etapa de limpeza de dados é muito importante. É uma etapa customizável, uma vez que podemos escolher diferentes estratégias de limpeza dos textos.
Aqui, escolhi o seguinte caminho:

- retirar acentuação e caractéres especiais;
- aplicar .lower() em todas as palavras;
- tokenização;
- remoção de stopwords da língua inglesa;
- aplicação de stemmer (extração dos núcleos das palavras).

In [11]:
def remove_special_char(input_string):
    return input_string.translate(str.maketrans('','', string.punctuation)).lower()

def preprocess_string(txt):
     
    # Removing special characters
    txt = remove_special_char(txt)

    # creating tokens
    tokens = nltk.tokenize.word_tokenize(txt) 
    
    # removing stopwords and applying stemmer
    tokens = [stemmer.stem(tk) for tk in tokens if tk not in stopwords]

    return tokens

In [12]:
# Creating the stemmer and stopwords list
stemmer = nltk.stem.PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')

# Applying preprocessing functions to documents and queries
document_corpus = [preprocess_string(txt) for txt in document_set.values()]
query_corpus = [preprocess_string(txt) for txt in query_set.values()]

A mesma limpeza que é aplicada aos documentos deve ser aplicada às queries.



# Testando Okapi BM25 

## Selecionando aleatoriamente um index para testar 

Vou escolher aleatoriamente um id para testar o funcionamento do BM25. 

In [14]:
# Randomly choosing an index do query
random.seed(42)
idx = random.sample(relevant_set.keys(),1)[0]

# Get query text
tokenized_query = query_corpus[idx]

# Get relevant documents
relevant_docs = relevant_set[idx] 

# Index all documents using BM25
bm25 = BM25Okapi(document_corpus)

# Calculating score and top N matches
scores = bm25.get_scores(tokenized_query)
topn = bm25.get_top_n(tokenized_query, document_corpus, n=10)

# printing outputs
print('Tokenized query:', tokenized_query)
print('Relevant documents IDs:', relevant_docs)
print('Top10 matches:', topn)
print('Scores:', scores)

Tokenized query: ['much', 'inform', 'retriev', 'dissemin', 'system', 'well', 'autom', 'librari', 'cost', 'worth', 'research', 'industri']
Relevant documents IDs: [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
Top10 matches: [['journal', 'select', 'model', 'implic', 'librari', 'system', 'kraft', 'h', 'hill', 'w', 'jr', 'problem', 'select', 'journal', 'acquir', 'order', 'best', 'satisfi', 'librari', 'object', 'model', 'zeroon', 'linear', 'program', 'problem', 'examin', 'detail', 'model', 'incorpor', 'object', 'function', 'base', 'expect', 'usag', 'measur', 'journal', 'worth', 'cost', 'constraint', 'account', 'scarciti', 'ca

Agora vou encapsular esse trecho de código em uma função para reaproveitar posteriormente e rodar para todas as queries.

In [15]:
def mean_reciprocal_rank(bool_results, k=10):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank

    ##Source: https://gist.github.com/bwhite/3726239
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

mean_reciprocal_rank([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

0.611111111111111

In [16]:
def results_from_query(tokenized_query, idx, bm25):
    """Return an ordered array of relevant documents returned by query_id

    Args:
        qry_id (int): id of query on dataset
        bm25 (object): indexed corpus

    Returns:
        boolean sorted relevance array of documents
    """    
    rel_docs = []

    # Getting only index that is in the relevant set
    if idx in relevant_set:
        rel_docs = relevant_set[idx]

    # Scoring query using BM25
    scores = bm25.get_scores(tokenized_query)

    # Getting most relevant documents
    most_relevant_documents = np.argsort([-1*x for x in scores])
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    # Setting 1 to relevant documents
    masked_relevance_results[rel_docs] = 1

    # Retrieving masked relevance results
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    
    return sorted_masked_relevance_results


results = [results_from_query(tokenized_query, idx, bm25) for idx, tokenized_query in enumerate(query_corpus)]
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.4459


# Implementação from scratch

Vou implementar no zero o BM25. Vou customizá-lo para que sua API siga o padrão da API do bm25Okapi, e assim testar ambos e compará-los.

In [17]:
class BM25:
    """
    Implementation of BM25 algorithm. It computes the term frequencies, document frequencies
    and scores for a given set of documents and query parameters.
    Methods:
        fit: given a set of corpus, it computes all the necessary statistics to 
            get bm25 matching scores 
        _score: computes a single score based on a query and an index
        get_scores: computes all scores based on a query and the given corpus

    """

    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        """
        Function that fits the statistics that are required to calculate BM25 ranking
        score using a given corpus.

        Args:
            corpus : list[list[str]]
                Each element in the list represents a document, and each document
                is a list of the terms.
        Returns:
            self
        """
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        # compute the inverse document frequency
        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def _score(self, query, index):
        """
        Function that computes a score based on the query and the index
        Args:
            query: user's query
            index: the index of the query
        Returns:
            score: score of the given query
        """
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

    def get_scores(self, query):
        """
        Function that get scores for all the documents in the corpus
        Args:
            query: input query
        Returns:
            scores: a list of all the scores
        """
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

# Função de extração de métricas

Irei utilizar a implementação da Huggingface da trec_eval, uma ferramenta de cálculo de métricas para sistemas de recuperação da informação. 

Para isso, foi necessário ajustar o formato do input para obedecer o formado esperado pela API do Huggingface.

In [18]:
def data_format_trec_eval(idx, relevant_docs, most_relevant_documents, scores):
    """
    Function that transforms a set of scores, retrieved and relevant documents
    into trec_eval format used by HuggingFace evaluate module.
    Args:
        idx: index of document
        relevant_docs: list of relevant documents
        most_relevant_documents: list of retrieved documents
        scores: list with BM25 scores
    Returns:
        qrel: dict with relevant documents in trec_eval format
        run: dict with retrieved docouments and scores in trec_eval format.
    ## SOURCE: https://huggingface.co/spaces/evaluate-metric/trec_eval
    """
    
    N = len(relevant_docs)

    qrel = {
        'query': [idx] * N,
        'q0': ['q0'] * N,
        "docid": [str(x) for x in relevant_docs],
        "rel": [str(x) for x in relevant_docs]
    }

    run = {
        "query": [idx] * N,
        "q0": ["q0"] * N,
        "docid": [str(x) for x in list(most_relevant_documents[:N])],
        "rank": list(range(N)),
        "score": sorted(scores)[::-1][:N],
        "system": ["test"] * N
    }

    return qrel, run


def extract_metrics(trec_eval, qrel, run):
    """
    Function that extract trec_eval metrics from qrel and run
    Args:
        trec_eval: object from HuggingFace lib responsible for calculate metrics.
        qrel: dict with relevant documents in trec_eval format
        run: dict with retrieved docouments and scores in trec_eval format
    Returns:
        metrics: dict with trec_eval metrics.
    """
    return trec_eval.compute(references=[qrel], predictions=[run])

Testando o pipeline para um id aleatório

In [19]:
# Randomly choosing an index do query
random.seed(42)
idx = random.sample(relevant_set.keys(),1)[0]

tokenized_query = query_corpus[idx] #get query text
relevant_docs = relevant_set[idx] #get relevant documents

# Fitting BM25 and recovering scores
bm25 = BM25()
bm25.fit(document_corpus)
scores = bm25.get_scores(tokenized_query)

# Getting most relevant documents
most_relevant_documents = np.argsort([-1*x for x in scores])
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[relevant_docs] = 1
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)

# trec_eval format
qrel, run = data_format_trec_eval(idx, relevant_docs, most_relevant_documents, scores)

# Getting metrics
trec_eval = load("trec_eval")
metrics = extract_metrics(trec_eval, qrel, run)

# print results
print('Tokenized query:', tokenized_query)
print('Relevant documents IDs:', relevant_docs)
print('Retrieved documents IDs:', most_relevant_documents[:82])
print('Intersection between relevant documents and retrieved documents:', set(relevant_docs).intersection(set(most_relevant_documents[:82])))
print('MRR@10', mean_reciprocal_rank([sorted_masked_relevance_results]))
print('Metrics:', metrics)

Tokenized query: ['much', 'inform', 'retriev', 'dissemin', 'system', 'well', 'autom', 'librari', 'cost', 'worth', 'research', 'industri']
Relevant documents IDs: [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
Retrieved documents IDs: [ 514  489 1263  176  590  254  978  859  374  864   16  848  313  888
  380  865  280 1297 1119    5 1276  335  614  610  405  653  420  286
  915 1077  177  387 1026   26 1052 1361  480  481  989 1138  473 1367
  571  727 1204 1113 1008  574  641  916  375   66  173 1257 1283 1150
 1409  174  134  838  953 1105  976 1124  339 1352  689  179  496  125
  287  294   73  800  985  675  873  320

# Encapsulando a lógica em função

Vou encapsular a lógica em uma função para ser capaz de executar para todas as queries.

In [20]:
def results_from_query(tokenized_query, idx, trec_eval, bm25):
    """Return an ordered array of relevant documents returned by query_id

    Args:
        tokenized_query: tokenized query to submit to BM25
        idx: index of tokenized query
        trec_eval: object to calculate metrics
        bm25: indexed corpus
    Returns:
        sorted_masked_relevance_results: sorted relevance array of documents
        metrics: trec_eval metrics
    """    
    relevant_docs = []

    # Retrieving relevant document
    if idx in relevant_set:
        relevant_docs = relevant_set[idx]

    # Scoring query using BM25
    scores = bm25.get_scores(tokenized_query)

    # Getting most relevant documents
    most_relevant_documents = np.argsort([-1*x for x in scores])
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    # Setting 1 to relevant documents
    masked_relevance_results[relevant_docs] = 1

    # Retrieving masked relevance results
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)

    # trec_eval format
    qrel, run = data_format_trec_eval(idx, relevant_docs, most_relevant_documents, scores)

    # Getting metrics
    metrics = extract_metrics(trec_eval, qrel, run)

    return sorted_masked_relevance_results, metrics


# Loading trec_eval
trec_eval = load("trec_eval")

# Running function to get results and metrics
output = [
    results_from_query(tokenized_query, idx, trec_eval, bm25) 
    for idx, tokenized_query in enumerate(query_corpus)
    if idx not in queries_with_no_relevant_docs
]

results = [x[0] for x in output]
metrics = [x[1] for x in output]

In [21]:
print('MRR@10 %.4f' % mean_reciprocal_rank(results))

MRR@10 0.6537


In [22]:
df_metrics = pd.DataFrame(metrics)
df_metrics.head()

,runid,num_ret,num_rel,num_rel_ret,num_q,map,gm_map,bpref,Rprec,recip_rank,...,P@1000,NDCG@5,NDCG@10,NDCG@15,NDCG@20,NDCG@30,NDCG@100,NDCG@200,NDCG@500,NDCG@1000
0,test,46,46,20,1,0.289305,0.289305,0.0,0.434783,1.0,...,0.020,0.253467,0.279872,0.326786,0.349504,0.396586,0.406102,0.406102,0.406102,0.406102
1,test,26,26,2,1,0.022575,0.022575,0.0,0.076923,0.5,...,0.002,0.075690,0.052630,0.042371,0.038037,0.082846,0.082846,0.082846,0.082846,0.082846
2,test,44,44,14,1,0.215630,0.215630,0.0,0.318182,1.0,...,0.014,0.622255,0.477933,0.451088,0.478047,0.453751,0.456523,0.456523,0.456523,0.456523
3,test,8,8,2,1,0.250000,0.250000,0.0,0.250000,1.0,...,0.002,0.470656,0.401536,0.401536,0.401536,0.401536,0.401536,0.401536,0.401536,0.401536
4,test,24,24,3,1,0.033730,0.033730,0.0,0.125000,0.5,...,0.003,0.127351,0.100872,0.117123,0.112292,0.117310,0.117310,0.117310,0.117310,0.117310


In [23]:
df_metrics.describe()

,num_ret,num_rel,num_rel_ret,num_q,map,gm_map,bpref,Rprec,recip_rank,P@5,...,P@1000,NDCG@5,NDCG@10,NDCG@15,NDCG@20,NDCG@30,NDCG@100,NDCG@200,NDCG@500,NDCG@1000
count,76.000000,76.000000,76.000000,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,...,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,40.973684,40.934211,11.460526,1.0,0.141416,0.141417,0.007185,0.246858,0.653509,0.423684,...,0.011461,0.251832,0.236444,0.241536,0.240273,0.238188,0.248046,0.250120,0.250120,0.250120
std,36.171434,36.153132,12.641140,0.0,0.130799,0.130798,0.035874,0.151907,0.383039,0.293879,...,0.012641,0.221134,0.189573,0.182704,0.181047,0.176827,0.173580,0.173783,0.173783,0.173783
min,1.000000,1.000000,0.000000,1.0,0.000000,0.000010,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,13.000000,3.000000,1.0,0.044356,0.044356,0.000000,0.145089,0.333333,0.200000,...,0.003000,0.068260,0.094246,0.101756,0.101913,0.106783,0.124962,0.124962,0.124962,0.124962
50%,30.500000,30.500000,8.000000,1.0,0.104906,0.104906,0.000000,0.250000,1.000000,0.400000,...,0.008000,0.213069,0.205570,0.204963,0.208783,0.208463,0.232966,0.232966,0.232966,0.232966
75%,52.250000,52.250000,15.000000,1.0,0.211609,0.211609,0.000000,0.351972,1.000000,0.600000,...,0.015000,0.396157,0.380725,0.385079,0.373350,0.350057,0.384257,0.384257,0.384257,0.384257
max,155.000000,155.000000,54.000000,1.0,0.688633,0.688633,0.205128,0.722222,1.000000,1.000000,...,0.054000,0.923995,0.804301,0.813944,0.785362,0.778038,0.778038,0.778038,0.778038,0.778038


Como pode ser observado, a média do `recip_rank` no `.describe()` do dataframe de métricas bate (aproximadamente) com o `MRR@10` calculado separadamente.

No próximo notebook vou encapsular todas as etapas deste notebook em uma classe única responsável pela lógica da Search Engine.